## Format dataset v4

In [1]:
import pandas as pd

In [2]:
new_df = pd.read_csv("./fine_tune_no_prompt_final.csv")

In [3]:
new_df.drop(columns=["Unnamed: 0"], inplace=True)

In [4]:
new_df.sample(10)

,category,context,question,response,instruction
2996,RAG,Butwal ma delivery normally 3-4 business days ...,Butwal ma delivery ma kati din lagchha?,Butwal ma delivery 3-4 business days ma hunchha.,Answer the following question by looking at th...
11754,alpaca,,Euta short description lekha about a dream vac...,Mero dream vacation Pokhara jane ho. Yaha ma P...,
6430,translate,,E-commerce ko uday le mancheharu ko kinbikri r...,The rise of e-commerce has changed the way peo...,Translate the text below from Nepali to English:
5690,RAG,Delivery Date Change Policy: Delivery date can...,Malai product ko delivery date change garna sa...,"Hajur, tapailai order ship huna agadi delivery...",Answer the following question by looking at th...
13787,open_qa,,Tyo kitaab ko lekhak ko naam ke ho?,Tyo kitaab ko lekhak Bishnu Prasad Koirala ho.,You are a helpful assistant. Answer the given ...
1453,open_qa,,Daal ko protin kasari protect garna sakincha?,Daal ko protin protect garna halka aang ma pak...,You are a helpful assistant. Answer the given ...
15022,RAG,Hamro delivery service Dharan vitra same day g...,delivery garne samay kati ho?,"Dharan vitra same day delivery garincha, bahir...",
17177,open_qa,,Tapai kasari Nepali font install garna saknuhu...,"Font file download garnus, double-click garera...",You are a helpful assistant. Answer the given ...
8113,alpaca,,Yedi timro sathi le timilai 'I forgot my homew...,"Ma uslai 'Teacher lai explain gara, ra feri na...",
1166,translate,,Aaja mero ghar ma sabai sanga milera khana kha...,Today I ate food with everyone at home. It fee...,Translate the text below from Nepali to English:


In [5]:
new_df.shape

(17866, 5)

In [6]:
new_df.isna().sum()

category       0
context        0
question       0
response       0
instruction    0
dtype: int64

## Format Prompt

In [7]:
def format_prompt_gemma(row):
# PROMPT FOR RAG DATASET
    if row["category"] == "RAG":
        prompt = f"""<start_of_turn>system
You are an AI assistant who responds to user instructions. Use the context information to answer if it is given, and respond in Roman Nepali.<end_of_turn>
<start_of_turn>user
{row["question"]}
Context: {row["context"]}<end_of_turn>
<start_of_turn>model
{row["response"]}"""
        return prompt

# PROMPT FOR OPEN QA IF CONTEXT IS GIVEN
    elif row["category"] == "open_qa" and row["context"] != " ":
        prompt = f"""<start_of_turn>system
You are helpful AI Assistant follow the user instruction and respond in Roman Nepali language.<end_of_turn>
<start_of_turn>user
{row["question"]}
Context: {row["context"]}<end_of_turn>
<start_of_turn>model
{row["response"]}"""
        return prompt
   
# PROMPT FOR OPEN QA IF CONTEXT IS NOT GIVEN 
    elif row["category"] == "open_qa" and row["context"] == " ":
        prompt = f"""<start_of_turn>system
You are helpful AI Assistant follow the user instruction and respond in Roman Nepali language.<end_of_turn>
<start_of_turn>user
{row["question"]}<end_of_turn>
<start_of_turn>model
{row["response"]}"""
        return prompt
 
# PROMPT FOR TRANSLATION   
    elif row["category"] == "translate":
        prompt = f"""<start_of_turn>system
You are helpful AI Assistant follow the user instruction.<end_of_turn>
<start_of_turn>user
{row["instruction"]}
{row["question"]}<end_of_turn>
<start_of_turn>model
{row["response"]}"""
        return prompt
 
# PROMPT FOR SUMMARIZATION   
    elif row["category"] == "summary":
        prompt = f"""<start_of_turn>system
You are helpful AI Assistant follow the user instruction.<end_of_turn>
<start_of_turn>user
{row["instruction"]}
{row["question"]}<end_of_turn>
<start_of_turn>model
{row["response"]}"""
        return prompt
    
# PROMPT FOR ALPACA IF CONTEXT IS NOT GIVEN
    elif row["category"] == "alpaca" and row["context"] == " ":
        prompt = f"""<start_of_turn>system
You are helpful AI Assistant follow the user instruction.<end_of_turn>
<start_of_turn>user
{row["question"]}<end_of_turn>
<start_of_turn>model
{row["response"]}"""
        return prompt
     
# PROMPT FOR ALPACA IF CONTEXT IS GIVEN   
    elif row["category"] == "alpaca" and row["context"] != " ":
        prompt = f"""<start_of_turn>system
You are helpful AI Assistant follow the user instruction and respond in Roman Nepali language.<end_of_turn>
<start_of_turn>user
{row["question"]}
Context: {row["context"]}<end_of_turn>
<start_of_turn>model
{row["response"]}"""
        return prompt

In [8]:
new_df["prompt"] = new_df.apply(format_prompt_gemma, axis=1)

In [9]:
print(new_df[new_df["category"] == "RAG"].iloc[0]["prompt"])

<start_of_turn>system
You are an AI assistant who responds to user instructions. Use the context information to answer if it is given, and respond in Roman Nepali.<end_of_turn>
<start_of_turn>user
Nepal ko sabai bhanda thulo mantralaya k ho?
Context: Nepal ko sabai bhanda thulo mantralaya Ministry of Home Affairs ho, jo Nepal ko sabai rajasik, suraksha ra lok prashasan ko kendra ho. Ministry of Home Affairs, jo Kathmandu ma sthit cha, Nepal ko rajnaitik aur vyavasthik gatividhi haru ko mahatvapurn ang ho. Yaha ko prashasan karyakram le Nepal ko sthirata aur suraksha ko sambhar garincha.<end_of_turn>
<start_of_turn>model
Ministry of Home Affairs, Nepal ko sabai bhanda thulo mantralaya ho, jo rajasik aur suraksha vyavastha ko kendra ho.


In [10]:
print(new_df[new_df["category"] == "open_qa"].iloc[0]["prompt"])

<start_of_turn>system
You are helpful AI Assistant follow the user instruction and respond in Roman Nepali language.<end_of_turn>
<start_of_turn>user
Timi afno manparne movie ko bare ma lekh.<end_of_turn>
<start_of_turn>model
Mero manparne movie 'Kabaddi' ho. Yo movie ma Nepal ko rural life ko taste ra character ko majaki herna majja lagcha. Sundar thau, ramailo story ra sabai character ma ek jindagi jasto feel cha.


In [11]:
print(new_df[new_df["category"] == "translate"].iloc[0]["prompt"])

<start_of_turn>system
You are helpful AI Assistant follow the user instruction.<end_of_turn>
<start_of_turn>user
Translate the text below from English to Nepali:
The concept of 'Atithi Devo Bhava' reflects the hospitality of Nepali people.<end_of_turn>
<start_of_turn>model
'Atithi Devo Bhava' ko concept le Nepali manche ko atithi satkar ko pratik garcha.


In [12]:
print(new_df[new_df["category"] == "summary"].iloc[0]["prompt"])

<start_of_turn>system
You are helpful AI Assistant follow the user instruction.<end_of_turn>
<start_of_turn>user
Summarize the following text:
Chitwan ma ek din ek narile gaun ko khel ma hissa liyin. Tyo khel gaun ko parampara ra samuhik jeevan ko sundar udaharan baneko thiyo.<end_of_turn>
<start_of_turn>model
Narile Chitwan ma parampara ko khel ma hissa liin.


In [13]:
print(new_df[new_df["category"] == "alpaca"].iloc[0]["prompt"])

<start_of_turn>system
You are helpful AI Assistant follow the user instruction.<end_of_turn>
<start_of_turn>user
Nepal ma mobile payment kasari popular bhaeko cha? Explain gara.<end_of_turn>
<start_of_turn>model
Nepal ma eSewa, Khalti jasta digital wallets le online payment easy banako cha. Pandemic pachhi digital transactions ko demand ajha badeko cha.


In [14]:
print(new_df[new_df["category"] == "alpaca"].iloc[100]["prompt"])

<start_of_turn>system
You are helpful AI Assistant follow the user instruction.<end_of_turn>
<start_of_turn>user
Yo question ko answer deu: 'Nepal ko national bird k ho?'<end_of_turn>
<start_of_turn>model
Nepal ko national bird Danphe ho.


In [15]:
new_df.to_csv("./fine_tune_gemma.csv", index=False)

In [16]:
new_df.shape

(17866, 6)

In [17]:
df_load_gemma_fine_tune = pd.read_csv("./fine_tune_gemma.csv")

In [18]:

df_load_gemma_fine_tune.shape

(17866, 6)

In [19]:
df_load_gemma_fine_tune.sample(5)

,category,context,question,response,instruction,prompt
10553,RAG,"Hamro store ma tablets, wireless chargers, ra ...",Hajurko laptops ma kehi discount chha?,Aaile laptops ma 10% discount cha.,Answer the following question by looking at th...,<start_of_turn>system\nYou are an AI assistant...
15078,alpaca,,Ekta short travel itinerary banauna help gara ...,1. Day 1: Phewa Lake ma boating gar ra Tal Bar...,,<start_of_turn>system\nYou are helpful AI Assi...
3254,translate,,Nepal mahaan biodiversity ko desh ho. Yo dhera...,Nepal is a country of great biodiversity. It i...,Translate the text below from Nepali to English:,<start_of_turn>system\nYou are helpful AI Assi...
8031,open_qa,,30 bata 12 ghatawa bhaye ko result kati ho?,30 bata 12 ghatawa bhaye ko result 18 ho.,You are a helpful assistant. Answer the given ...,<start_of_turn>system\nYou are helpful AI Assi...
17573,RAG,"Product available: Nike Running Shoes, Price: ...",Nike Running Shoes ko price kati ho?,Nike Running Shoes ko price Rs. 5000 ho.,Answer the following question by looking at th...,<start_of_turn>system\nYou are an AI assistant...
